In [1]:
import pandas as pd
import esm

In [2]:
# codes adapted from https://github.com/facebookresearch/esm'
# put model in eval mode: no parameter updating
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()
model.eval()

ProteinBertModel(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj):

In [3]:
# use A0A0A0MT70 as an example
data = [
    ('protein1', 'SGANVLTFGAGSRLTVL')
]
seq = list('SGANVLTFGAGSRLTVL')

In [4]:
import torch
batch_labels, batch_strs, batch_tokens = batch_converter(data)
with torch.no_grad():
    output = model(batch_tokens, repr_layers=[33], return_contacts = True)
# original output shape ([1,19,33]), transpose it to ([1, 33, 19])
# to match up the matrix size given in the paper
logit_output = output['logits'].permute(0, 2, 1)

In [5]:
# check size
logit_output.size()

torch.Size([1, 33, 19])

In [6]:
# handle data in the pandas dataframe (easier than handle tensor)
csv_ = pd.DataFrame(logit_output[0])
csv_

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,21.934128,-13.903990,-14.663964,-15.523140,-15.130095,-15.310490,-15.232232,-13.611775,-14.137093,-14.624403,-15.498966,-14.606729,-15.148205,-14.499645,-14.767580,-13.393473,-15.178053,-15.014832,-5.729622
1,-7.111636,-16.928429,-18.180866,-17.026180,-18.401276,-17.668518,-18.344379,-18.443905,-17.929802,-18.164005,-16.913763,-17.998381,-17.532227,-18.536476,-17.966303,-18.167004,-17.553532,-18.041470,-13.709194
2,6.767663,-7.957729,-11.822146,-12.050881,-11.145052,-12.037618,-11.531459,-11.174477,-10.593109,-12.144470,-12.386029,-11.708278,-9.464024,-11.359426,-11.182956,-10.788742,-11.887415,-11.051668,32.664291
3,-7.061787,-16.963173,-18.245090,-17.001995,-18.471497,-17.567272,-18.364990,-18.390427,-18.002899,-18.253059,-16.951015,-18.094376,-17.533518,-18.538483,-17.925240,-18.114639,-17.463047,-18.009314,-13.810287
4,-5.486125,-0.552936,-0.371572,-0.488361,-0.374301,-0.346139,3.306105,-0.211891,0.532455,-0.378623,-0.569122,-0.411388,-0.450525,-0.332905,3.238981,-0.196143,-0.138591,2.710788,-1.355098
5,-4.990091,-0.777761,-0.170209,2.820541,-0.100401,-0.285153,-0.340986,-0.186864,0.206251,-0.138256,3.013323,-0.264826,-0.250784,-0.342844,-0.540325,-0.406477,-0.374920,-0.324806,-0.355509
6,-5.548675,-0.621321,3.145092,-0.171093,-0.119890,-0.286215,-0.607176,-0.388111,-0.152113,2.930521,-0.462319,3.230002,-0.020210,-0.248697,-0.523822,-0.581112,-0.427013,-0.305036,-0.948963
7,-6.467486,-0.953361,-0.599864,-0.670216,-0.426107,2.942108,-0.068541,-0.226274,0.551149,-0.685391,-0.767379,-0.460472,-0.501295,-0.370811,-0.181268,-0.431346,3.040008,-0.682857,-1.577004
8,-5.948035,2.283210,-0.068819,-0.203472,-0.281341,-0.150003,-0.439703,-0.190293,-0.132388,-0.082769,-0.068097,-0.275258,3.010210,-0.025076,-0.466055,-0.317361,-0.375320,-0.061517,-0.891256
9,-6.675358,-1.759497,-0.855966,-0.712262,-0.859478,-0.884535,-1.161020,-0.740297,-1.174551,-0.879325,-0.938112,-1.111248,-0.646835,-0.989708,-1.074070,-0.732845,-1.304816,-0.626064,-2.034184


In [13]:
csv_copy = csv_.copy()

In [14]:
# vocalbulary dictionary
# matches tokens to amino acids
voc_dict = {
            alphabet.tok_to_idx[tok] : tok
            for tok in alphabet.tok_to_idx
            if '<' not in tok and tok != '.' and tok != '-'
        }
voc_dict

{4: 'L',
 5: 'A',
 6: 'G',
 7: 'V',
 8: 'S',
 9: 'E',
 10: 'R',
 11: 'T',
 12: 'I',
 13: 'D',
 14: 'P',
 15: 'K',
 16: 'Q',
 17: 'N',
 18: 'F',
 19: 'Y',
 20: 'M',
 21: 'H',
 22: 'W',
 23: 'C',
 24: 'X',
 25: 'B',
 26: 'U',
 27: 'Z',
 28: 'O'}

In [15]:
# drop rows (unused vocalbulary, like <unk>, and unusual amino acids like
# B,O,U,Z,X
# I tried not to drop the unusual amino acids, but the results won't match
csv_copy.drop([0,1,2,3,24,25,26,27,28,29,30,31,32], inplace=True)

In [16]:
# drop columns, the <cls> and <eos>, hard coded by select the column index to drop
csv_copy.drop(columns=[0,18], inplace=True)
# rename the column by wt seq
csv_copy.columns = seq
# rename the row name by the vocalbulary dictionary
csv_copy = csv_copy.rename(index=voc_dict)
csv_copy

,S,G,A,N,V,L,T,F,G,A,G,S,R,L,T,V,L
L,-0.552936,-0.371572,-0.488361,-0.374301,-0.346139,3.306105,-0.211891,0.532455,-0.378623,-0.569122,-0.411388,-0.450525,-0.332905,3.238981,-0.196143,-0.138591,2.710788
A,-0.777761,-0.170209,2.820541,-0.100401,-0.285153,-0.340986,-0.186864,0.206251,-0.138256,3.013323,-0.264826,-0.250784,-0.342844,-0.540325,-0.406477,-0.374920,-0.324806
G,-0.621321,3.145092,-0.171093,-0.119890,-0.286215,-0.607176,-0.388111,-0.152113,2.930521,-0.462319,3.230002,-0.020210,-0.248697,-0.523822,-0.581112,-0.427013,-0.305036
V,-0.953361,-0.599864,-0.670216,-0.426107,2.942108,-0.068541,-0.226274,0.551149,-0.685391,-0.767379,-0.460472,-0.501295,-0.370811,-0.181268,-0.431346,3.040008,-0.682857
S,2.283210,-0.068819,-0.203472,-0.281341,-0.150003,-0.439703,-0.190293,-0.132388,-0.082769,-0.068097,-0.275258,3.010210,-0.025076,-0.466055,-0.317361,-0.375320,-0.061517
E,-1.759497,-0.855966,-0.712262,-0.859478,-0.884535,-1.161020,-0.740297,-1.174551,-0.879325,-0.938112,-1.111248,-0.646835,-0.989708,-1.074070,-0.732845,-1.304816,-0.626064
R,-1.392528,-0.749550,-0.559157,-0.641854,-0.562175,-0.758772,-0.421823,-0.767007,-0.760692,-0.720848,-0.792966,-0.594253,2.623814,-0.736485,-0.515160,-0.691315,-0.458055
T,-1.209871,-0.397357,-0.616455,-0.510158,-0.040049,-0.282298,3.105240,-0.369889,-0.356797,-0.508554,-0.636733,-0.485523,-0.050194,-0.555353,3.026469,-0.605783,-0.271215
I,-1.467153,-1.000715,-1.086500,-1.230406,-0.890204,-0.553593,-0.745320,0.160009,-1.096523,-1.088328,-0.884286,-1.062197,-0.925710,-0.511231,-0.870360,-0.525886,-1.250720
D,-1.053338,-0.767459,-0.722513,-0.841973,-0.948753,-1.297071,-0.551693,-0.722460,-0.852384,-0.336327,-0.675162,-0.743310,-0.972222,-1.380167,-1.054980,-1.269209,-1.023354


In [18]:
# apply softmax (since need to take log, cannot apply log to negative values)
import numpy as np
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0) # apply to each column
csv_copy_softmax = csv_copy.transform(softmax,axis=0)

In [19]:
# double check softmax is correct
csv_copy_softmax.sum(axis=0)
# get 17 1s, means probability of aa per residue summed up to 1

S    1.0
G    1.0
A    1.0
N    1.0
V    1.0
L    1.0
T    1.0
F    1.0
G    1.0
A    1.0
G    1.0
S    1.0
R    1.0
L    1.0
T    1.0
V    1.0
L    1.0
dtype: float32

In [20]:
# check the paper's output
df = pd.read_csv('~/A0A0A0MT70_LLR.csv', index_col=0)
df

,S 1,G 2,A 3,N 4,V 5,L 6,T 7,F 8,G 9,A 10,G 11,S 12,R 13,L 14,T 15,V 16,L 17
K,-4.346,-4.114,-3.604,-3.121,-3.967,-4.549,-4.081,-3.798,-3.872,-3.918,-4.397,-3.999,-3.475,-4.349,-3.858,-4.195,-3.552
R,-3.676,-3.895,-3.380,-2.693,-3.504,-4.065,-3.527,-3.251,-3.691,-3.734,-4.023,-3.604,0.000,-3.975,-3.542,-3.731,-3.169
H,-4.660,-4.981,-4.587,-3.886,-4.693,-5.255,-4.500,-4.245,-4.691,-4.776,-5.056,-4.663,-4.454,-5.243,-4.643,-4.951,-4.583
E,-4.043,-4.001,-3.533,-2.911,-3.827,-4.467,-3.846,-3.659,-3.810,-3.951,-4.341,-3.657,-3.614,-4.313,-3.759,-4.345,-3.337
D,-3.337,-3.913,-3.543,-2.894,-3.891,-4.603,-3.657,-3.207,-3.783,-3.350,-3.905,-3.754,-3.596,-4.619,-4.081,-4.309,-3.734
N,-3.848,-4.186,-3.656,0.000,-4.012,-4.733,-4.046,-3.455,-3.988,-3.647,-4.019,-3.978,-3.557,-4.497,-4.206,-4.315,-3.640
Q,-4.225,-4.261,-3.896,-3.207,-3.956,-4.606,-4.065,-3.759,-3.990,-4.215,-4.560,-3.874,-3.728,-4.549,-3.891,-4.377,-3.704
T,-3.493,-3.542,-3.437,-2.562,-2.982,-3.588,0.000,-2.854,-3.287,-3.522,-3.867,-3.496,-2.674,-3.794,0.000,-3.646,-2.982
S,0.000,-3.214,-3.024,-2.333,-3.092,-3.746,-3.296,-2.616,-3.013,-3.081,-3.505,0.000,-2.649,-3.705,-3.344,-3.415,-2.772
C,-4.689,-5.111,-4.723,-4.005,-4.821,-5.284,-4.910,-4.070,-4.875,-4.956,-5.230,-4.742,-4.566,-4.967,-4.895,-4.844,-4.720


In [21]:
# the original number took the natural log, so i transform it back
df_log = np.exp(df)
df_log

,S 1,G 2,A 3,N 4,V 5,L 6,T 7,F 8,G 9,A 10,G 11,S 12,R 13,L 14,T 15,V 16,L 17
K,0.012959,0.016342,0.027215,0.044113,0.018930,0.010578,0.016891,0.022416,0.020817,0.019881,0.012314,0.018334,0.030962,0.012920,0.021110,0.015071,0.028667
R,0.025324,0.020343,0.034047,0.067678,0.030077,0.017163,0.029393,0.038735,0.024947,0.023897,0.017899,0.027215,1.000000,0.018779,0.028955,0.023969,0.042046
H,0.009466,0.006867,0.010183,0.020527,0.009159,0.005221,0.011109,0.014336,0.009178,0.008430,0.006371,0.009438,0.011632,0.005284,0.009629,0.007076,0.010224
E,0.017545,0.018297,0.029217,0.054421,0.021775,0.011482,0.021365,0.025758,0.022148,0.019235,0.013023,0.025810,0.026944,0.013393,0.023307,0.012972,0.035543
D,0.035543,0.019980,0.028926,0.055354,0.020425,0.010022,0.025810,0.040478,0.022754,0.035084,0.020141,0.023424,0.027433,0.009863,0.016891,0.013447,0.023897
N,0.021322,0.015207,0.025836,1.000000,0.018097,0.008800,0.017492,0.031587,0.018537,0.026069,0.017971,0.018723,0.028524,0.011142,0.014906,0.013367,0.026252
Q,0.014625,0.014108,0.020323,0.040478,0.019140,0.009992,0.017163,0.023307,0.018500,0.014772,0.010462,0.020775,0.024041,0.010578,0.020425,0.012563,0.024625
T,0.030410,0.028955,0.032161,0.077150,0.050691,0.027654,1.000000,0.057613,0.037366,0.029540,0.020921,0.030318,0.068976,0.022505,1.000000,0.026095,0.050691
S,1.000000,0.040196,0.048606,0.097004,0.045411,0.023612,0.037031,0.073095,0.049144,0.045913,0.030047,1.000000,0.070722,0.024600,0.035295,0.032876,0.062537
C,0.009196,0.006030,0.008888,0.018224,0.008059,0.005072,0.007372,0.017077,0.007635,0.007041,0.005354,0.008721,0.010399,0.006964,0.007484,0.007875,0.008915


In [ ]:
# take the first entry, S1K as example, the entry is 0.012959
# csv_copy_softmax, in the first column, entry K has 0.006878
# in csv_copy_softmax, divide the K by entry S (0.530243), will get
# 0.01297141, which is close enough for 0.012959
# so the transformation is like this: takes ratio, softmax, then natural log

In [26]:
# take ratio
csv_copy_softmax_copy = csv_copy_softmax.copy()

In [27]:
csv_copy_softmax_copy_take_ratio = csv_copy_softmax_copy / csv_copy_softmax_copy.max()
csv_copy_softmax_copy_take_ratio

,S,G,A,N,V,L,T,F,G,A,G,S,R,L,T,V,L
L,0.058651,0.029698,0.036556,0.088404,0.037319,1.000000,0.036257,0.142040,0.036547,0.027808,0.026216,0.031407,0.051989,1.000000,0.039851,0.041644,1.000000
A,0.046842,0.036323,1.000000,0.116259,0.039666,0.026067,0.037176,0.102505,0.046478,1.000000,0.030354,0.038350,0.051475,0.022839,0.032292,0.032879,0.048046
G,0.054774,1.000000,0.050205,0.114015,0.039624,0.019975,0.030399,0.071632,1.000000,0.030942,1.000000,0.048295,0.056557,0.023219,0.027117,0.031210,0.049005
V,0.039298,0.023637,0.030478,0.083941,1.000000,0.034230,0.035739,0.144721,0.026892,0.022807,0.024960,0.029852,0.050055,0.032704,0.031499,1.000000,0.033586
S,1.000000,0.040199,0.048606,0.097017,0.045406,0.023617,0.037048,0.073059,0.049130,0.045894,0.030039,1.000000,0.070730,0.024599,0.035301,0.032866,0.062518
E,0.017550,0.018296,0.029223,0.054421,0.021783,0.011480,0.021375,0.025767,0.022152,0.019227,0.013020,0.025809,0.026957,0.013393,0.023300,0.012974,0.035549
R,0.025331,0.020351,0.034058,0.067652,0.030068,0.017165,0.029391,0.038731,0.024942,0.023893,0.017900,0.027202,1.000000,0.018771,0.028966,0.023961,0.042052
T,0.030407,0.028942,0.032161,0.077174,0.050683,0.027642,1.000000,0.057614,0.037354,0.029544,0.020927,0.030327,0.068975,0.022498,1.000000,0.026101,0.050691
I,0.023509,0.015831,0.020100,0.037555,0.021659,0.021074,0.021268,0.097873,0.017827,0.016545,0.016338,0.017036,0.028738,0.023513,0.020306,0.028272,0.019034
D,0.035559,0.019989,0.028925,0.055382,0.020428,0.010020,0.025812,0.040496,0.022756,0.035097,0.020138,0.023435,0.027432,0.009861,0.016883,0.013444,0.023894


In [28]:
# take log
csv_copy_softmax_copy_take_ratio_and_log = np.log(csv_copy_softmax_copy_take_ratio)
csv_copy_softmax_copy_take_ratio_and_log

,S,G,A,N,V,L,T,F,G,A,G,S,R,L,T,V,L
L,-2.836146,-3.516663,-3.308902,-2.425833,-3.288247,0.000000,-3.317131,-1.951644,-3.309144,-3.582445,-3.641390,-3.460735,-2.956720,0.000000,-3.222612,-3.178598,0.000000
A,-3.060971,-3.315300,0.000000,-2.151933,-3.227262,-3.647091,-3.292104,-2.277847,-3.068777,0.000000,-3.494828,-3.260993,-2.966658,-3.779305,-3.432947,-3.414928,-3.035594
G,-2.904531,0.000000,-2.991634,-2.171422,-3.228323,-3.913280,-3.493351,-2.636211,0.000000,-3.475642,0.000000,-3.030419,-2.872511,-3.762803,-3.607582,-3.467021,-3.015824
V,-3.236571,-3.744956,-3.490757,-2.477640,0.000000,-3.374645,-3.331514,-1.932949,-3.615912,-3.780702,-3.690474,-3.511505,-2.994625,-3.420249,-3.457815,0.000000,-3.393645
S,0.000000,-3.213911,-3.024013,-2.332873,-3.092111,-3.745808,-3.295532,-2.616487,-3.013290,-3.081420,-3.505260,0.000000,-2.648890,-3.705036,-3.343830,-3.415328,-2.772305
E,-4.042707,-4.001057,-3.532803,-2.911010,-3.826643,-4.467125,-3.845537,-3.658649,-3.809846,-3.951435,-4.341249,-3.657044,-3.613522,-4.313051,-3.759314,-4.344823,-3.336852
R,-3.675738,-3.894641,-3.379698,-2.693386,-3.504283,-4.064877,-3.527062,-3.251106,-3.691213,-3.734171,-4.022967,-3.604463,0.000000,-3.975466,-3.541629,-3.731322,-3.168843
T,-3.493081,-3.542449,-3.436996,-2.561690,-2.982157,-3.588402,0.000000,-2.853987,-3.287318,-3.521877,-3.866734,-3.495733,-2.674008,-3.794334,0.000000,-3.645790,-2.982003
I,-3.750363,-4.145806,-3.907041,-3.281938,-3.832312,-3.859698,-3.850559,-2.324089,-4.027044,-4.101651,-4.114287,-4.072406,-3.549524,-3.750212,-3.896829,-3.565894,-3.961508
D,-3.336548,-3.912551,-3.543054,-2.893505,-3.890862,-4.603176,-3.656933,-3.206558,-3.782905,-3.349650,-3.905164,-3.753520,-3.596036,-4.619148,-4.081449,-4.309216,-3.734142


In [30]:
# reorder
original_order = list(df_log.index)
original_order

['K',
 'R',
 'H',
 'E',
 'D',
 'N',
 'Q',
 'T',
 'S',
 'C',
 'G',
 'A',
 'V',
 'L',
 'I',
 'M',
 'P',
 'Y',
 'F',
 'W']

In [33]:
csv_copy_softmax_copy_take_ratio_and_log_re_order = csv_copy_softmax_copy_take_ratio_and_log.loc[original_order,:]
csv_copy_softmax_copy_take_ratio_and_log_re_order

,S,G,A,N,V,L,T,F,G,A,G,S,R,L,T,V,L
K,-4.346406,-4.113620,-3.604161,-3.120601,-3.966984,-4.549388,-4.081175,-3.797895,-3.871937,-3.917914,-4.397108,-3.998513,-3.474555,-4.349486,-3.857910,-4.194745,-3.551680
R,-3.675738,-3.894641,-3.379698,-2.693386,-3.504283,-4.064877,-3.527062,-3.251106,-3.691213,-3.734171,-4.022967,-3.604463,0.000000,-3.975466,-3.541629,-3.731322,-3.168843
H,-4.659587,-4.981018,-4.587331,-3.886271,-4.692606,-5.255467,-4.499883,-4.244685,-4.691226,-4.775589,-5.055776,-4.662626,-4.454354,-5.243009,-4.643389,-4.951308,-4.582925
E,-4.042707,-4.001057,-3.532803,-2.911010,-3.826643,-4.467125,-3.845537,-3.658649,-3.809846,-3.951435,-4.341249,-3.657044,-3.613522,-4.313051,-3.759314,-4.344823,-3.336852
D,-3.336548,-3.912551,-3.543054,-2.893505,-3.890862,-4.603176,-3.656933,-3.206558,-3.782905,-3.349650,-3.905164,-3.753520,-3.596036,-4.619148,-4.081449,-4.309216,-3.734142
N,-3.848253,-4.186154,-3.656331,0.000000,-4.011615,-4.732526,-4.045895,-3.455359,-3.988105,-3.646531,-4.018928,-3.977586,-3.556970,-4.496571,-4.205689,-4.314854,-3.639809
Q,-4.225114,-4.260530,-3.896398,-3.206881,-3.956425,-4.605908,-4.065274,-3.758713,-3.990394,-4.215295,-4.560456,-3.874282,-3.727796,-4.549333,-3.890916,-4.376652,-3.703959
T,-3.493081,-3.542449,-3.436996,-2.561690,-2.982157,-3.588402,0.000000,-2.853987,-3.287318,-3.521877,-3.866734,-3.495733,-2.674008,-3.794334,0.000000,-3.645790,-2.982003
S,0.000000,-3.213911,-3.024013,-2.332873,-3.092111,-3.745808,-3.295532,-2.616487,-3.013290,-3.081420,-3.505260,0.000000,-2.648890,-3.705036,-3.343830,-3.415328,-2.772305
C,-4.688942,-5.111134,-4.722771,-4.005383,-4.820930,-5.284466,-4.909753,-4.070023,-4.874504,-4.956016,-5.229678,-4.741736,-4.565964,-4.966785,-4.895387,-4.843682,-4.719964


In [34]:
# save the csv
csv_copy_softmax_copy_take_ratio_and_log_re_order.to_csv('A0A0A0MT70_reconstruct.csv')